# Data Gathering
This notebook will gather data for Python mapping exercise.<br>
The idea will be to explore different mapping technologies in the context of Python using <b>NYC Open Data</b> and <b>US Census American Community Survey</b> data.

Will use <a href="https://pypi.org/project/census/>">census package</a> in Python to get:
- Population
- Median Age
- Median Income
- Poverty Level - will develop percent poverty level
- Households with car - will develop percent with vehicle
- Education - will develop percent college degree
- Racial - will develop percent race

We will work with the `state_zipcode()` function within the `census` package.<br>
We can grab <a href="https://data.cityofnewyork.us/Health/Modified-Zip-Code-Tabulation-Areas-MODZCTA-/pri4-ifjk/about_data">MODZCTA shapefiles</a> from NYC Open data, and list the ZCTA to pull the census data with the API.<br>
We will rename the Census variables accordingly, develop the percent features and merge with the GeoDataFrame.

Then we can get to mapping!!!

In [15]:
# import packages
import pandas as pd
import geopandas as gpd
import logging
import census
from us import states

# import census api key
from src.config import CENSUS_API

## Load Data
### MODZCTA Data
We need to download the shapefile from NYC Open data for mapping, as well as construct a ZCTA list to pass into the census api.

In [16]:
# load downloaded shapefile
# can write code to programmatically download this after, not important right now
gdf=gpd.read_file("./data/shp/Modified Zip Code Tabulation Areas (MODZCTA)_20240418/geo_export_bdb2fc16-3964-47c7-a04d-4d106b707aaf.shp")
# format column names
gdf.columns = [col.lower() for col in gdf.columns]
# preview
gdf.head()

,modzcta,label,zcta,pop_est,geometry
0,10001,"10001, 10118","10001, 10119, 10199",23072.0,"POLYGON ((-73.98774 40.74407, -73.98819 40.743..."
1,10002,10002,10002,74993.0,"POLYGON ((-73.99750 40.71407, -73.99709 40.714..."
2,10003,10003,10003,54682.0,"POLYGON ((-73.98864 40.72293, -73.98876 40.722..."
3,10026,10026,10026,39363.0,"MULTIPOLYGON (((-73.96201 40.80551, -73.96007 ..."
4,10004,10004,10004,3028.0,"MULTIPOLYGON (((-74.00827 40.70772, -74.00937 ..."


In [17]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   modzcta   178 non-null    object  
 1   label     177 non-null    object  
 2   zcta      178 non-null    object  
 3   pop_est   178 non-null    float64 
 4   geometry  178 non-null    geometry
dtypes: float64(1), geometry(1), object(3)
memory usage: 7.1+ KB


Let's grab the list of zip codes for NYC from the shapefile.<br>
We will need these to pass into the census api.

In [18]:
# create zip_list
zip_list=list(gdf['zcta'].str.split(',').explode())
# remove leading spaces
zip_list=[z.replace(' ','') for z in zip_list]

### Census Data

In [19]:
# set api key
c = census.Census(CENSUS_API)

We will construct a dictionary of variables with the desired column name as the key, and the actual census variable name as the value.<br>
This will allow us to easily rename the columns.

In [20]:
# dictionary of variables
var_dict = {
    'name': 'NAME',
    'population': 'B03002_001E',
    'median_age': 'B01002_001E',
    'median_household_income': 'B19013_001E',
    'poverty_level': 'B17001_002E',
    'white': 'B03002_003E',
    'black': 'B03002_004E',
    'american_indian_alaskan': 'B03002_005E',
    'asian': 'B03002_006E',
    'nhpi': 'B03002_007E',
    'other': 'B03002_008E',
    'two_or_more': 'B03002_009E',
    'hispanic': 'B03002_012E',
    'total_households': 'B08201_001E',
    'total_households_no_vehicle': 'B08201_002E',
    'pop_25_older': 'B15003_001E',
    'pop_25_older_hs_grad': 'B15003_017E',
    'pop_25_older_associates': 'B15003_019E',
    'pop_25_older_bachelors': 'B15003_020E',
    'pop_25_older_graduate': 'B15003_021E',
}

# get list of values for api call
variables=list(var_dict.values())

# define other search variables
# state fips
ny_fips=states.NY.fips
# year
year = 2020

The census api can only call 50 at a time, so we will loop through the zip codes to get data for all.

In [21]:
# # initialize list
# all_data = []

# # api call
# try:
#   for zip_code in zip_list:
#     data=c.acs5.state_zipcode(fields=variables,
#                       state_fips=ny_fips,
#                       year=year,
#                       zcta=zip_code)
#     all_data.extend(data)
# except KeyError as e:
#   logging.error(f"KeyError for ZIP code {zip_code}: {e}")
# except Exception as e:
#   logging.error(f"Unexpected error for ZIP code {zip_code}: {e}")

Let's create dataframe.

In [22]:
# # create dataframe
# df_acs_2020=pd.DataFrame(all_data)
# df_acs_2020.head()

Export df to avoid calling api in future

In [23]:
# df_acs_2020.to_pickle("./data/df_acs_2020.pkl")

## START ITERATING OVER CODE HERE, DON'T RUN API OVER AND OVER*

In [24]:
# import pickled df
df_acs_2020=pd.read_pickle("./data/df_acs_2020.pkl")

In [25]:
df_acs_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   NAME                      214 non-null    object 
 1   B03002_001E               214 non-null    float64
 2   B01002_001E               214 non-null    float64
 3   B19013_001E               214 non-null    float64
 4   B17001_002E               214 non-null    float64
 5   B03002_003E               214 non-null    float64
 6   B03002_004E               214 non-null    float64
 7   B03002_005E               214 non-null    float64
 8   B03002_006E               214 non-null    float64
 9   B03002_007E               214 non-null    float64
 10  B03002_008E               214 non-null    float64
 11  B03002_009E               214 non-null    float64
 12  B03002_012E               214 non-null    float64
 13  B08201_001E               214 non-null    float64
 14  B08201_002

In [26]:
# remove last col
df_acs_2020=df_acs_2020.iloc[:,:-1]

In [27]:
# rename columns
df_acs_2020.columns = var_dict.keys()

In [28]:
df_acs_2020.head()

,name,population,median_age,median_household_income,poverty_level,white,black,american_indian_alaskan,asian,nhpi,other,two_or_more,hispanic,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate
0,ZCTA5 10001,25026.0,36.1,96787.0,2798.0,13641.0,1536.0,11.0,5201.0,63.0,107.0,542.0,3925.0,13311.0,11290.0,19550.0,1307.0,427.0,1004.0,579.0
1,ZCTA5 10119,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ZCTA5 10199,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ZCTA5 10002,74363.0,44.8,35607.0,20257.0,16476.0,5776.0,375.0,31011.0,0.0,388.0,1182.0,19155.0,33790.0,28446.0,58942.0,8972.0,1293.0,3897.0,2459.0
4,ZCTA5 10003,54671.0,31.9,129981.0,4040.0,37168.0,2738.0,38.0,8238.0,69.0,146.0,1542.0,4732.0,25158.0,19715.0,38411.0,1710.0,476.0,1661.0,1408.0


## Data Cleaning
1) Remove `ZCTA` from `name` column, and rename as `zcta`
2) Rename name to `zcta`


In [29]:
# remove ZCTA from name col
df_acs_2020['name']=[name[6:] for name in df_acs_2020['name']]

In [30]:
# rename name to zcta
df_acs_2020.rename(columns={'name':'zcta'}, inplace=True)

In [31]:
df_acs_2020.head()

,zcta,population,median_age,median_household_income,poverty_level,white,black,american_indian_alaskan,asian,nhpi,other,two_or_more,hispanic,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate
0,10001,25026.0,36.1,96787.0,2798.0,13641.0,1536.0,11.0,5201.0,63.0,107.0,542.0,3925.0,13311.0,11290.0,19550.0,1307.0,427.0,1004.0,579.0
1,10119,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10199,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10002,74363.0,44.8,35607.0,20257.0,16476.0,5776.0,375.0,31011.0,0.0,388.0,1182.0,19155.0,33790.0,28446.0,58942.0,8972.0,1293.0,3897.0,2459.0
4,10003,54671.0,31.9,129981.0,4040.0,37168.0,2738.0,38.0,8238.0,69.0,146.0,1542.0,4732.0,25158.0,19715.0,38411.0,1710.0,476.0,1661.0,1408.0


Why do we have some -66666666.0 values?<br>
Does this have something to do with the secondary zip code set?<br>
Let's extract them and find out.

In [32]:
# function to extract secondary zip
def extract_secondary_zip(zip_codes):
  zip_list = zip_codes.strip().split(',')
  return [zip_code.strip() for zip_code in zip_list[1:]]

# apply function
gdf['secondary_zip']=gdf['zcta'].apply(extract_secondary_zip)

# set of zips that are secondary
secondary_zip_set = set(zip_code for zip_codes in gdf['secondary_zip'] for zip_code in zip_codes)

Let's check to see how the `secondary_zip` column handles those that do not have secondary zips.

In [33]:
gdf[gdf['modzcta'] == '10002']

,modzcta,label,zcta,pop_est,geometry,secondary_zip
1,10002,10002,10002,74993.0,"POLYGON ((-73.99750 40.71407, -73.99709 40.714...",[]


Well, it just creates a blank list. I guess that is fine?

Let's try joining `df_acs_2020` to `gdf`. We will have to do a few things first:

- If there is a record in `secondary_zip` column, then copy the row and create a new one
- Iterate over each `secondary_zip` if there is more then one so that there is a row with all the same info

In [34]:
new_records = []  # List to store new records

for idx, row in gdf.iterrows():
    primary_zip = row['zcta'].split(',')[0].strip()  # Extract primary zip code
    secondary_zips = row['secondary_zip']
    
    # If there are secondary zip codes, create new records
    if secondary_zips:
        for sec_zip in secondary_zips:
            # Create a copy of the row and update the 'zcta' column with the secondary zip
            new_row = row.copy()
            new_row['zcta'] = sec_zip.strip()
            new_records.append(new_row)

# Create a new DataFrame from the list of new records
new_df = pd.DataFrame(new_records)

Ok, now let's concatenate it to the `gdf` and remove the secondary zips from the original `zcta` records

In [18]:
# concatenate gdf with secondary zip code df
gdf_full=pd.concat([gdf,new_df],ignore_index=True)
# drop anything after primary in zcta column
for idx, row in gdf_full.iterrows():
  gdf_full.loc[idx,'zcta'] = row['zcta'].split(',')[0].strip()
# drop secondary zip col
gdf_full.drop(columns='secondary_zip',inplace=True)


In [19]:
gdf_full.head()

,modzcta,label,zcta,pop_est,geometry
0,10001,"10001, 10118",10001,23072.0,"POLYGON ((-73.98774 40.74407, -73.98819 40.743..."
1,10002,10002,10002,74993.0,"POLYGON ((-73.99750 40.71407, -73.99709 40.714..."
2,10003,10003,10003,54682.0,"POLYGON ((-73.98864 40.72293, -73.98876 40.722..."
3,10026,10026,10026,39363.0,"MULTIPOLYGON (((-73.96201 40.80551, -73.96007 ..."
4,10004,10004,10004,3028.0,"MULTIPOLYGON (((-74.00827 40.70772, -74.00937 ..."


In [23]:
gdf_full.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   modzcta   215 non-null    object  
 1   label     214 non-null    object  
 2   zcta      215 non-null    object  
 3   pop_est   215 non-null    float64 
 4   geometry  215 non-null    geometry
dtypes: float64(1), geometry(1), object(3)
memory usage: 8.5+ KB


`gdf_full` has 215 records (or zip codes) and `df_acs_2020` has 214. Thats better then prior, but I wonder what the one extra is.<br>
There is one without a label, let's check that one out.

In [25]:
gdf_full[gdf_full['label'].isna()]

,modzcta,label,zcta,pop_est,geometry
177,99999,NaN,99999,0.0,"MULTIPOLYGON (((-74.21417 40.55659, -74.21409 ..."


Ah yes, the old 99999 zip code, how does this even exist? Does it exist in the original `gdf`?

In [26]:
gdf[gdf['modzcta'] == '99999']

,modzcta,label,zcta,pop_est,geometry,secondary_zip
177,99999,NaN,99999,0.0,"MULTIPOLYGON (((-74.21417 40.55659, -74.21409 ...",[]


What about in the `df_acs_2020`?

In [27]:
df_acs_2020[df_acs_2020['zcta'] == '99999']

,zcta,population,median_age,median_household_income,poverty_level,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate


Ok, its only in the `gdf`, I am going to drop it.

In [28]:
# drop 99999 from gdf_full
gdf_full=gdf_full[gdf_full['modzcta'] != '99999']

In [29]:
# check info
gdf_full.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 214 entries, 0 to 214
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   modzcta   214 non-null    object  
 1   label     214 non-null    object  
 2   zcta      214 non-null    object  
 3   pop_est   214 non-null    float64 
 4   geometry  214 non-null    geometry
dtypes: float64(1), geometry(1), object(3)
memory usage: 10.0+ KB



Ok good, now we have a redundant `gdf_full` with exactly 214 zip codes that we can join the `df_acs_2020` to, and then groupby by `modzcta`.<br>
This should allow us to account for all zip codes and aggregate values accordingly.

In [20]:
gdf_full.merge(df_acs_2020, right_on='zcta', left_on='zcta', how='left')

,modzcta,label,zcta,pop_est,geometry,population,median_age,median_household_income,poverty_level,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate
0,10001,"10001, 10118",10001,23072.0,"POLYGON ((-73.98774 40.74407, -73.98819 40.743...",25026.0,36.1,96787.0,2798.0,13311.0,11290.0,19550.0,1307.0,427.0,1004.0,579.0
1,10002,10002,10002,74993.0,"POLYGON ((-73.99750 40.71407, -73.99709 40.714...",74363.0,44.8,35607.0,20257.0,33790.0,28446.0,58942.0,8972.0,1293.0,3897.0,2459.0
2,10003,10003,10003,54682.0,"POLYGON ((-73.98864 40.72293, -73.98876 40.722...",54671.0,31.9,129981.0,4040.0,25158.0,19715.0,38411.0,1710.0,476.0,1661.0,1408.0
3,10026,10026,10026,39363.0,"MULTIPOLYGON (((-73.96201 40.80551, -73.96007 ...",38937.0,35.4,64716.0,7921.0,15362.0,11413.0,27757.0,4583.0,681.0,2989.0,1137.0
4,10004,10004,10004,3028.0,"MULTIPOLYGON (((-74.00827 40.70772, -74.00937 ...",3310.0,38.4,204949.0,93.0,1822.0,1347.0,2899.0,55.0,2.0,77.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,11369,11369,11371,34118.0,"POLYGON ((-73.88258 40.75585, -73.88296 40.757...",0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211,11411,11411,11411,20930.0,"POLYGON ((-73.73222 40.68512, -73.73309 40.685...",20473.0,45.1,104269.0,768.0,6183.0,732.0,15295.0,3272.0,958.0,2660.0,1543.0
212,11429,11429,11429,31780.0,"MULTIPOLYGON (((-73.71050 40.72723, -73.71051 ...",27808.0,40.9,82532.0,2448.0,8006.0,1737.0,19941.0,3830.0,793.0,3295.0,1824.0
213,11433,11433,11451,36489.0,"POLYGON ((-73.79437 40.68691, -73.79478 40.687...",0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df_acs_2020[df_acs_2020['zcta'] == '11451']

,zcta,population,median_age,median_household_income,poverty_level,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate
208,11451,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# view zip codes in secondary zip set
df_acs_2020[df_acs_2020['zcta'].isin(secondary_zip_set)]

,zcta,population,median_age,median_household_income,poverty_level,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate
1,10119,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10199,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,10271,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,10278,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,10279,96.0,-666666666.0,-666666666.0,0.0,40.0,40.0,96.0,0.0,0.0,56.0,0.0
21,10165,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,10167,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,10168,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,10169,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,10170,0.0,-666666666.0,-666666666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


If we isolate the zip codes that were combined for each MODZCTA, we can see many of them have 0.0 population and bizarre negative values for other Census metrics. Let's drop any records that have -666666666.0.

Ok, let's first create a dictionary mapping secondary zips to their primary, and if there isn't one primary to primary.<br>
This way we can create a column in the acs data as primary zip, and group accordingly.

In [25]:
# create a dictionary mapping secondary zip codes to their corresponding primary
zip_mapping={}
for idx, row in gdf.iterrows():
  primary_zip = row['zcta'].split(',')[0].strip() # strip blank space in front of first zip
  secondary_zips = row['zcta'].split(',')[1:] # all secondary zips for each record
  for zip_code in secondary_zips:
    zip_mapping[zip_code.strip()] = primary_zip
  if not secondary_zips:
    zip_mapping[primary_zip] = primary_zip

# create new column in acs data with primary zip code
df_acs_2020['modzcta'] = df_acs_2020['zcta'].apply(lambda x: zip_mapping.get(x.strip()))

In [26]:
# drop zip codes with zero population
df_acs_2020=df_acs_2020[df_acs_2020['population'] != 0]

Check secondary zip code list after dropping 0 population

In [27]:
df_acs_2020[df_acs_2020['zcta'].isin(secondary_zip_set)]

,zcta,population,median_age,median_household_income,poverty_level,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate,modzcta
12,10279,96.0,-666666666.0,-666666666.0,0.0,40.0,40.0,96.0,0.0,0.0,56.0,0.0,10007
68,10162,1240.0,40.1,96555.0,0.0,622.0,405.0,904.0,230.0,0.0,0.0,56.0,10075
85,10314,89938.0,41.7,90306.0,8109.0,31363.0,3502.0,63621.0,17385.0,3250.0,7453.0,3988.0,10311
97,11005,2249.0,85.1,75742.0,202.0,1609.0,447.0,2249.0,300.0,59.0,425.0,0.0,11004
98,11040,41523.0,43.9,132767.0,1359.0,13078.0,634.0,29547.0,5802.0,869.0,2846.0,1885.0,11004
146,11424,40.0,42.5,-666666666.0,0.0,0.0,0.0,26.0,13.0,0.0,0.0,0.0,11415
162,11357,40118.0,46.8,82858.0,2343.0,14437.0,1542.0,29968.0,6680.0,1462.0,3232.0,2039.0,11351
164,11360,18892.0,50.9,84356.0,1015.0,8293.0,1368.0,14856.0,2666.0,689.0,1469.0,1016.0,11359
189,11411,20473.0,45.1,104269.0,768.0,6183.0,732.0,15295.0,3272.0,958.0,2660.0,1543.0,11003
205,11429,27808.0,40.9,82532.0,2448.0,8006.0,1737.0,19941.0,3830.0,793.0,3295.0,1824.0,11001


Ok, still two with -666666666.0 in `median_household_income`, let's drop those records as well.

In [28]:
# drop -666666666.0 in median_househould_income
df_acs_2020=df_acs_2020[df_acs_2020['median_household_income'] != -666666666.0]

Check secondary zip code set once more.

In [29]:
df_acs_2020[df_acs_2020['zcta'].isin(secondary_zip_set)]

,zcta,population,median_age,median_household_income,poverty_level,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate,modzcta
68,10162,1240.0,40.1,96555.0,0.0,622.0,405.0,904.0,230.0,0.0,0.0,56.0,10075
85,10314,89938.0,41.7,90306.0,8109.0,31363.0,3502.0,63621.0,17385.0,3250.0,7453.0,3988.0,10311
97,11005,2249.0,85.1,75742.0,202.0,1609.0,447.0,2249.0,300.0,59.0,425.0,0.0,11004
98,11040,41523.0,43.9,132767.0,1359.0,13078.0,634.0,29547.0,5802.0,869.0,2846.0,1885.0,11004
162,11357,40118.0,46.8,82858.0,2343.0,14437.0,1542.0,29968.0,6680.0,1462.0,3232.0,2039.0,11351
164,11360,18892.0,50.9,84356.0,1015.0,8293.0,1368.0,14856.0,2666.0,689.0,1469.0,1016.0,11359
189,11411,20473.0,45.1,104269.0,768.0,6183.0,732.0,15295.0,3272.0,958.0,2660.0,1543.0,11003
205,11429,27808.0,40.9,82532.0,2448.0,8006.0,1737.0,19941.0,3830.0,793.0,3295.0,1824.0,11001
210,11434,62590.0,38.9,65845.0,5849.0,21510.0,5882.0,43069.0,10990.0,2330.0,8967.0,3392.0,11430


Ok, these all look reasonable, will have to find out a way to combine them with the primary zip code identified in the `gdf`, as these would otherwise be unaccounted for if we just chose to gather data for the main zip code in each MODZCTA from the gdf file.

In [30]:
# group by primary zip, reset index, drop zcta
df_acs_2020_cleaned=df_acs_2020.groupby('modzcta').agg({'zcta':'first',
                                                            'population':'sum',
                                                            'median_age':'median', 
                                                            'median_household_income':'median',
                                                            'poverty_level':'sum', 
                                                            'total_households':'sum',
                                                            'total_households_no_vehicle':'sum',
                                                            'pop_25_older':'sum', 
                                                            'pop_25_older_hs_grad':'sum', 
                                                            'pop_25_older_associates':'sum',
                                                            'pop_25_older_bachelors':'sum',
                                                            'pop_25_older_graduate':'sum'}).reset_index()

In [31]:
pd.set_option('display.max_rows',None)
df_acs_2020_cleaned.head()

,modzcta,zcta,population,median_age,median_household_income,poverty_level,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate
0,10002,10002,74363.0,44.8,35607.0,20257.0,33790.0,28446.0,58942.0,8972.0,1293.0,3897.0,2459.0
1,10003,10003,54671.0,31.9,129981.0,4040.0,25158.0,19715.0,38411.0,1710.0,476.0,1661.0,1408.0
2,10004,10004,3310.0,38.4,204949.0,93.0,1822.0,1347.0,2899.0,55.0,2.0,77.0,22.0
3,10006,10006,3260.0,33.1,185268.0,205.0,1811.0,1563.0,2675.0,48.0,0.0,105.0,31.0
4,10009,10009,58267.0,37.0,68220.0,14134.0,29356.0,23264.0,46500.0,5062.0,1152.0,3148.0,1844.0


In [32]:
df_acs_2020_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   modzcta                      165 non-null    object 
 1   zcta                         165 non-null    object 
 2   population                   165 non-null    float64
 3   median_age                   165 non-null    float64
 4   median_household_income      165 non-null    float64
 5   poverty_level                165 non-null    float64
 6   total_households             165 non-null    float64
 7   total_households_no_vehicle  165 non-null    float64
 8   pop_25_older                 165 non-null    float64
 9   pop_25_older_hs_grad         165 non-null    float64
 10  pop_25_older_associates      165 non-null    float64
 11  pop_25_older_bachelors       165 non-null    float64
 12  pop_25_older_graduate        165 non-null    float64
dtypes: float64(11), obje

In [33]:
df_acs_2020.info()

<class 'pandas.core.frame.DataFrame'>
Index: 182 entries, 0 to 213
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   zcta                         182 non-null    object 
 1   population                   182 non-null    float64
 2   median_age                   182 non-null    float64
 3   median_household_income      182 non-null    float64
 4   poverty_level                182 non-null    float64
 5   total_households             182 non-null    float64
 6   total_households_no_vehicle  182 non-null    float64
 7   pop_25_older                 182 non-null    float64
 8   pop_25_older_hs_grad         182 non-null    float64
 9   pop_25_older_associates      182 non-null    float64
 10  pop_25_older_bachelors       182 non-null    float64
 11  pop_25_older_graduate        182 non-null    float64
 12  modzcta                      166 non-null    object 
dtypes: float64(11), object(2)

`df_acs_2020_cleaned` has 165 records while the MODZCTA `gdf` has 178, let's see which are mmissing.

In [34]:
# extract unique values
gdf_modzcta=set(gdf['modzcta'])
df_modzcta_cleaned=set(df_acs_2020_cleaned['modzcta'])

# find values in gdf that are not present in df
missing_modzcta=gdf_modzcta-df_modzcta_cleaned

# print
print(f"There are {len(missing_modzcta)}, they are:")
print(missing_modzcta)

There are 19, they are:
{'99999', '10022', '11415', '11209', '10001', '11411', '10005', '11434', '11429', '11369', '10019', '10017', '10007', '11357', '11433', '10027', '10314', '10036', '11360'}


Is there a difference between `df_acs_2020` and the `gdf`?

In [35]:
# extract unique values
df_modzcta=set(df_acs_2020['zcta'])

# find values in gdf that are not present in df
missing_modzcta_org=gdf_modzcta-df_modzcta

# print
print(f"There are {len(missing_modzcta_org)}, they are:")
print(missing_modzcta_org)

There are 1, they are:
{'99999'}


Ok, it looks like the groupby is whats removing these zip codes, let's look at the code that forms the `modzcta` column in the `df_acs_2020` and see if we can adjust so these remain.

Ok, can we see if these were in the intial data?

In [36]:
# filter df_acs_2020 with this list
df_acs_2020[df_acs_2020['zcta'].isin(list(missing_modzcta))]

,zcta,population,median_age,median_household_income,poverty_level,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate,modzcta
0,10001,25026.0,36.1,96787.0,2798.0,13311.0,11290.0,19550.0,1307.0,427.0,1004.0,579.0,None
7,10005,8664.0,30.4,184681.0,653.0,4649.0,4256.0,6698.0,278.0,3.0,76.0,28.0,None
10,10007,7566.0,34.4,250001.0,100.0,2940.0,2262.0,5676.0,214.0,116.0,80.0,84.0,None
20,10017,16065.0,40.9,130417.0,1138.0,9858.0,7681.0,13656.0,388.0,397.0,505.0,79.0,None
33,10019,45521.0,39.5,101651.0,5389.0,27340.0,22561.0,39713.0,3263.0,959.0,2385.0,1932.0,None
39,10022,31574.0,48.1,138661.0,1581.0,17977.0,13034.0,26977.0,1326.0,412.0,1201.0,856.0,None
46,10027,64728.0,31.1,57010.0,13049.0,23835.0,18572.0,42603.0,7197.0,556.0,4477.0,2054.0,None
56,10036,28231.0,37.7,97720.0,3715.0,17437.0,15323.0,23433.0,1510.0,574.0,1311.0,991.0,None
85,10314,89938.0,41.7,90306.0,8109.0,31363.0,3502.0,63621.0,17385.0,3250.0,7453.0,3988.0,10311
123,11209,68368.0,40.6,79524.0,7291.0,29186.0,13428.0,50594.0,8003.0,2018.0,5013.0,2957.0,None


Yes they are, this means that somehow they were lost during aggregation.<br>
Will have to figure out why.<br>
Are they in the gdf?

In [37]:
gdf[gdf['modzcta'].isin(missing_modzcta)]

,modzcta,label,zcta,pop_est,geometry,secondary_zip
0,10001,"10001, 10118","10001, 10119, 10199",23072.0,"POLYGON ((-73.98774 40.74407, -73.98819 40.743...","[10119, 10199]"
5,10005,10005,"10005, 10271",8831.0,"POLYGON ((-74.00783 40.70309, -74.00786 40.703...",[10271]
7,10007,10007,"10007, 10278, 10279",7023.0,"POLYGON ((-74.00998 40.70981, -74.01123 40.710...","[10278, 10279]"
15,10017,10017,"10017, 10165, 10167, 10168, 10169, 10170, 1017...",15801.0,"POLYGON ((-73.98182 40.75219, -73.98137 40.752...","[10165, 10167, 10168, 10169, 10170, 10171, 101..."
18,10019,"10019, 10020","10019, 10020, 10103, 10111, 10112",43703.0,"POLYGON ((-74.00115 40.76203, -74.00128 40.762...","[10020, 10103, 10111, 10112]"
20,10022,10022,"10022, 10152, 10153, 10154",31125.0,"POLYGON ((-73.97766 40.75791, -73.97720 40.758...","[10152, 10153, 10154]"
24,10027,10027,"10027, 10115",64548.0,"POLYGON ((-73.96581 40.80884, -73.96545 40.809...",[10115]
33,10036,10036,"10036, 10110",27428.0,"POLYGON ((-74.00115 40.76203, -74.00039 40.762...",[10110]
59,10314,10314,"10311, 10314",90761.0,"POLYGON ((-74.16968 40.56647, -74.16976 40.566...",[10314]
95,11209,11209,"11209, 11425",67809.0,"POLYGON ((-74.01942 40.60287, -74.01975 40.603...",[11425]


Oddly they are, and the common denominator is they are all primary zip codes in the `modzcta` column.<br>
Perhaps there is something wrong with the zip_mapping dictionary code.

## Feature Creation

Need to create a few features that will make it easier to visualize data.

1) `% poverty level` - take poverty level number and divide by population
2) `% households with car` - take households without a car, subtract from households, then divide result by households
3) `% college degree` - sum up associates degree and higher, divide this number by total population 25 and older 
4) Remove `ZCTA` from `name` column, and rename as `zcta`

## Feature Creation

Need to create a few features that will make it easier to visualize data.

1) `% poverty level` - take poverty level number and divide by population
2) `% households with car` - take households without a car, subtract from households, then divide result by households
3) `% college degree` - sum up associates degree and higher, divide this number by total population 25 and older and remove columns. 
4) `other_race` - create columne by summing up 5, 7-11
5) `% race columns` - create perc rate columns for each
6) Drop uneeded columns
7) Reorder columns


In [38]:
# create % poverty level
df_acs_2020_cleaned['perc_poverty_level'] = df_acs_2020_cleaned['poverty_level'] / df_acs_2020_cleaned['population']

In [39]:
# create % with car
df_acs_2020_cleaned['perc_hh_w_vehicle'] = \
(df_acs_2020_cleaned['total_households'] - \
 df_acs_2020_cleaned['total_households_no_vehicle']) / df_acs_2020_cleaned['total_households']

In [40]:
# create % college degree
df_acs_2020_cleaned['perc_college_degree'] = \
    (df_acs_2020_cleaned['pop_25_older_associates'] + \
     df_acs_2020_cleaned['pop_25_older_bachelors'] + df_acs_2020_cleaned['pop_25_older_graduate']) / df_acs_2020['pop_25_older']

In [41]:
pd.set_option('display.max_columns',None)
df_acs_2020_cleaned.head()

,modzcta,population,median_age,median_household_income,poverty_level,total_households,total_households_no_vehicle,pop_25_older,pop_25_older_hs_grad,pop_25_older_associates,pop_25_older_bachelors,pop_25_older_graduate,perc_poverty_level,perc_hh_w_vehicle,perc_college_degree
0,10075,1240.0,40.1,96555.0,0.0,622.0,405.0,904.0,230.0,0.0,0.0,56.0,0.000000,0.348875,0.002864
1,10311,89938.0,41.7,90306.0,8109.0,31363.0,3502.0,63621.0,17385.0,3250.0,7453.0,3988.0,0.090162,0.888340,NaN
2,11001,27808.0,40.9,82532.0,2448.0,8006.0,1737.0,19941.0,3830.0,793.0,3295.0,1824.0,0.088032,0.783038,NaN
3,11003,20473.0,45.1,104269.0,768.0,6183.0,732.0,15295.0,3272.0,958.0,2660.0,1543.0,0.037513,0.881611,0.087561
4,11004,43772.0,64.5,104254.5,1561.0,14687.0,1081.0,31796.0,6102.0,928.0,3271.0,1885.0,0.035662,0.926397,0.158392


In [42]:
df_acs_2020_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   modzcta                      8 non-null      object 
 1   population                   8 non-null      float64
 2   median_age                   8 non-null      float64
 3   median_household_income      8 non-null      float64
 4   poverty_level                8 non-null      float64
 5   total_households             8 non-null      float64
 6   total_households_no_vehicle  8 non-null      float64
 7   pop_25_older                 8 non-null      float64
 8   pop_25_older_hs_grad         8 non-null      float64
 9   pop_25_older_associates      8 non-null      float64
 10  pop_25_older_bachelors       8 non-null      float64
 11  pop_25_older_graduate        8 non-null      float64
 12  perc_poverty_level           8 non-null      float64
 13  perc_hh_w_vehicle       

Export df for mapping notebook.

In [53]:
df_acs_2020.to_pickle("./data/df_acs_2020_cleaned.pkl")